# Cloning the Metric3D Repository from Hugging Face

In [1]:
# Before running the Kaggle notebook, make sure to select the GPU P100 under the 'Accelerator' option in the 'Session Options'

## all install

In [2]:
!pip install mmengine pyngrok plyfile html4vision open3d --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.9.2 which is incompatible.


In [3]:
!rm -rf /kaggle/working/Texture_training


In [4]:
! git clone https://github.com/Pritipaudel/Texture_training.git

Cloning into 'Texture_training'...
remote: Enumerating objects: 222, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 222 (delta 91), reused 212 (delta 81), pack-reused 0 (from 0)
Receiving objects: 100% (222/222), 226.34 KiB | 8.38 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [5]:
%cd /kaggle/working/Texture_training

/kaggle/working/Texture_training


---

In [6]:

!git fetch origin
!git checkout AO

Branch 'AO' set up to track remote branch 'AO' from 'origin'.
Switched to a new branch 'AO'


In [7]:
%cd /kaggle/working/Texture_training/training

/kaggle/working/Texture_training/training


In [8]:
import os
import torch
import wandb
from mmengine import Config
from mono.model.monodepth_model import DepthModel
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from PIL import Image
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import pandas as pd
from kaggle_secrets import UserSecretsClient
from huggingface_hub import HfApi, HfFolder, Repository, create_repo, upload_file
from huggingface_hub import login

# Installing Essential Dependencies and Resolving Version Conflicts for 3D Monodepth

In [9]:
!pip install  pyngrok
!ngrok authtoken 2tCJTC3fdY5MfELkvtwaIM2XyQU_4pjEPWHJPFgPVR74ySQrs

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:
path = "/kaggle/working/Texture_training"
cfg = Config.fromfile(path+'/training/mono/configs/RAFTDecoder/vit.raft5.small.py')

# Initialize the DepthModel
model = DepthModel(cfg, None)

In [11]:
from huggingface_hub import hf_hub_download

# Download the model file from Hugging Face Hub
repo_name = "pbr_extractor"
downloaded_file = hf_hub_download(
    repo_id=f"preetipaudel/{repo_name}",  # Replace with your Hugging Face username
    filename="AO_depth_connected_1781.pth"
)
print(f"Model downloaded from Hugging Face Hub: {downloaded_file}")



# Initialize the model and load the state_dict

model.load_state_dict(torch.load(downloaded_file),strict=False)
model.eval()  # Set to evaluation mode
print("Model loaded successfully from Hugging Face Hub!")

AO_depth_connected_1781.pth:   0%|          | 0.00/154M [00:00<?, ?B/s]

Model downloaded from Hugging Face Hub: /root/.cache/huggingface/hub/models--preetipaudel--pbr_extractor/snapshots/119a9ee54cb87ef5ee8ec9c40922cda5dffc8d88/AO_depth_connected_1781.pth


/tmp/ipykernel_23/2658076220.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(downloaded_file),strict=False)


Model loaded successfully from Hugging Face Hub!


---

# Importing Required Libraries and Modules for 3D Monodepth and Visualization

In [12]:
%%writefile /kaggle/working/Texture_training/training/mono/utils/do_test.py
import torch
import torch.nn.functional as F
import logging
import os
import cv2
from tqdm import tqdm
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# from mono.utils.unproj_pcd import reconstruct_pcd, save_point_cloud

def to_cuda(data: dict):
    for k, v in data.items():
        if isinstance(v, torch.Tensor):
            data[k] = v.cuda(non_blocking=True)
        if isinstance(v, list) and len(v)>=1 and isinstance(v[0], torch.Tensor):
            for i, l_i in enumerate(v):
                data[k][i] = l_i.cuda(non_blocking=True)
    return data

def align_scale(pred: torch.tensor, target: torch.tensor):
    mask = target > 0
    if torch.sum(mask) > 10:
        scale = torch.median(target[mask]) / (torch.median(pred[mask]) + 1e-8)
    else:
        scale = 1
    pred_scaled = pred * scale
    return pred_scaled, scale

def align_scale_shift(pred: torch.tensor, target: torch.tensor):
    mask = target > 0
    target_mask = target[mask].cpu().numpy()
    pred_mask = pred[mask].cpu().numpy()
    if torch.sum(mask) > 10:
        scale, shift = np.polyfit(pred_mask, target_mask, deg=1)
        if scale < 0:
            scale = torch.median(target[mask]) / (torch.median(pred[mask]) + 1e-8)
            shift = 0
    else:
        scale = 1
        shift = 0
    pred = pred * scale + shift
    return pred, scale

def align_scale_shift_numpy(pred: np.array, target: np.array):
    mask = target > 0
    target_mask = target[mask]
    pred_mask = pred[mask]
    if np.sum(mask) > 10:
        scale, shift = np.polyfit(pred_mask, target_mask, deg=1)
        if scale < 0:
            scale = np.median(target[mask]) / (np.median(pred[mask]) + 1e-8)
            shift = 0
    else:
        scale = 1
        shift = 0
    pred = pred * scale + shift
    return pred, scale


def build_camera_model(H : int, W : int, intrinsics : list) -> np.array:
    """
    Encode the camera intrinsic parameters (focal length and principle point) to a 4-channel map. 
    """
    fx, fy, u0, v0 = intrinsics
    f = (fx + fy) / 2.0
    # principle point location
    x_row = np.arange(0, W).astype(np.float32)
    x_row_center_norm = (x_row - u0) / W
    x_center = np.tile(x_row_center_norm, (H, 1)) # [H, W]

    y_col = np.arange(0, H).astype(np.float32) 
    y_col_center_norm = (y_col - v0) / H
    y_center = np.tile(y_col_center_norm, (W, 1)).T # [H, W]

    # FoV
    fov_x = np.arctan(x_center / (f / W))
    fov_y = np.arctan(y_center / (f / H))

    cam_model = np.stack([x_center, y_center, fov_x, fov_y], axis=2)
    return cam_model

def resize_for_input(image, output_shape, intrinsic, canonical_shape, to_canonical_ratio):
    """
    Resize the input.
    Resizing consists of two processed, i.e. 1) to the canonical space (adjust the camera model); 2) resize the image while the camera model holds. Thus the
    label will be scaled with the resize factor.
    """
    padding = [123.675, 116.28, 103.53]
    h, w, _ = image.shape
    resize_ratio_h = output_shape[0] / canonical_shape[0]
    resize_ratio_w = output_shape[1] / canonical_shape[1]
    to_scale_ratio = min(resize_ratio_h, resize_ratio_w)

    resize_ratio = to_canonical_ratio * to_scale_ratio

    reshape_h = int(resize_ratio * h)
    reshape_w = int(resize_ratio * w)

    pad_h = max(output_shape[0] - reshape_h, 0)
    pad_w = max(output_shape[1] - reshape_w, 0)
    pad_h_half = int(pad_h / 2)
    pad_w_half = int(pad_w / 2)

    # resize
    image = cv2.resize(image, dsize=(reshape_w, reshape_h), interpolation=cv2.INTER_LINEAR)
    # padding
    image = cv2.copyMakeBorder(
        image, 
        pad_h_half, 
        pad_h - pad_h_half, 
        pad_w_half, 
        pad_w - pad_w_half, 
        cv2.BORDER_CONSTANT, 
        value=padding)
    
    # Resize, adjust principle point
    intrinsic[2] = intrinsic[2] * to_scale_ratio
    intrinsic[3] = intrinsic[3] * to_scale_ratio

    cam_model = build_camera_model(reshape_h, reshape_w, intrinsic)
    cam_model = cv2.copyMakeBorder(
        cam_model, 
        pad_h_half, 
        pad_h - pad_h_half, 
        pad_w_half, 
        pad_w - pad_w_half, 
        cv2.BORDER_CONSTANT, 
        value=-1)

    pad=[pad_h_half, pad_h - pad_h_half, pad_w_half, pad_w - pad_w_half]
    label_scale_factor=1/to_scale_ratio
    return image, cam_model, pad, label_scale_factor


def get_prediction(
    model: torch.nn.Module,
    input: torch.tensor,
    cam_model: torch.tensor,
    pad_info: torch.tensor,
    scale_info: torch.tensor,
    gt_depth: torch.tensor,
    normalize_scale: float,
    ori_shape: list=[],
):

    data = dict(
        input=input,
        cam_model=cam_model,
    )
    #pred_depth, confidence, output_dict = model.module.inference(data)
    prediction = model.inference(data)
    pred_depth=prediction["prediction"]
    
    pred_depth = pred_depth.squeeze()
    pred_depth = pred_depth[pad_info[0] : pred_depth.shape[0] - pad_info[1], pad_info[2] : pred_depth.shape[1] - pad_info[3]]
    # confidence = confidence.squeeze()
    # confidence = confidence[pad_info[0] : confidence.shape[0] - pad_info[1], pad_info[2] : confidence.shape[1] - pad_info[3]]
    if gt_depth is not None:
        resize_shape = gt_depth.shape
    elif ori_shape != []:
        resize_shape = ori_shape
    else:
        resize_shape = pred_depth.shape

    pred_depth = torch.nn.functional.interpolate(pred_depth[None, None, :, :], resize_shape, mode='bilinear').squeeze() # to original size
    pred_depth = pred_depth * normalize_scale / scale_info
    if gt_depth is not None:
        pred_depth_scale, scale = align_scale(pred_depth, gt_depth)
    else:
        pred_depth_scale = None
        scale = None
    confidence=[]
    return pred_depth, pred_depth_scale, scale, prediction, confidence

def transform_test_data_scalecano(rgb, intrinsic, data_basic):
    """
    Pre-process the input for forwarding. Employ `label scale canonical transformation.'
        Args:
            rgb: input rgb image. [H, W, 3]
            intrinsic: camera intrinsic parameter, [fx, fy, u0, v0]
            data_basic: predefined canonical space in configs.
    """
    canonical_space = data_basic['canonical_space']
    forward_size = data_basic.crop_size
    mean = torch.tensor([123.675, 116.28, 103.53]).float()[:, None, None]
    std = torch.tensor([58.395, 57.12, 57.375]).float()[:, None, None]

    # BGR to RGB
    rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)

    ori_h, ori_w, _ = rgb.shape
    ori_focal = (intrinsic[0] + intrinsic[1]) / 2
    canonical_focal = canonical_space['focal_length']

    cano_label_scale_ratio = canonical_focal / ori_focal

    canonical_intrinsic = [
        intrinsic[0] * cano_label_scale_ratio,
        intrinsic[1] * cano_label_scale_ratio,
        intrinsic[2],
        intrinsic[3],
    ]

    # resize
    rgb, cam_model, pad, resize_label_scale_ratio = resize_for_input(rgb, forward_size, canonical_intrinsic, [ori_h, ori_w], 1.0)

    # label scale factor
    label_scale_factor = cano_label_scale_ratio * resize_label_scale_ratio

    rgb = torch.from_numpy(rgb.transpose((2, 0, 1))).float()
    rgb = torch.div((rgb - mean), std)
    rgb = rgb[None, :, :, :].cuda()
    #rgb = rgb[None, :, :, :]
    
    cam_model = torch.from_numpy(cam_model.transpose((2, 0, 1))).float()
    cam_model = cam_model[None, :, :, :].cuda()
    #cam_model = cam_model[None, :, :, :]
    cam_model_stacks = [
        torch.nn.functional.interpolate(cam_model, size=(cam_model.shape[2]//i, cam_model.shape[3]//i), mode='bilinear', align_corners=False)
        for i in [2, 4, 8, 16, 32]
    ]
    return rgb, cam_model_stacks, pad, label_scale_factor







Overwriting /kaggle/working/Texture_training/training/mono/utils/do_test.py


In [13]:
import torch
import torch.nn.functional as F
import logging
import os
import os.path as osp
import cupy
import sys
try:
    from mmcv.utils import Config, DictAction
except:
    from mmengine import Config, DictAction

import glob
# from mono.utils.comm import init_env
# from mono.model.monodepth_model import get_configured_monodepth_model
# from mono.utils.running import load_ckpt
from mono.utils.do_test import transform_test_data_scalecano, get_prediction
# from mono.utils.custom_data import load_from_annos, load_data

# from mono.utils.avg_meter import MetricAverageMeter
# from mono.utils.visualization import save_val_imgs, create_html, save_raw_imgs, save_normal_val_imgs
# import cv2
# from tqdm import tqdm
# import numpy as np
# from PIL import Image, ExifTags
# import matplotlib.pyplot as plt

from mono.utils.unproj_pcd import reconstruct_pcd, save_point_cloud #, ply_to_obj
from mono.utils.transform import gray_to_colormap
from mono.utils.visualization import vis_surface_normal
# import gradio as gr
# import plotly.graph_objects as go
from pyngrok import ngrok
import cv2

---

# Main Code Segment

In [14]:
!mkdir /kaggle/working/processed_folder

In [15]:
# cfg_large = Config.fromfile('/kaggle/working/Metric3D/mono/configs/HourglassDecoder/vit.raft5.large.py')
# model_large = get_configured_monodepth_model(cfg_large, )
# model_large, _,  _, _ = load_ckpt('/kaggle/working/Metric3D/weight/metric_depth_vit_large_800k.pth', model_large, strict_match=False)
# model_large.eval()

# cfg_small = Config.fromfile('/kaggle/working/Metric3D/mono/configs/HourglassDecoder/vit.raft5.small.py')
# model_small = get_configured_monodepth_model(cfg_small, )
# model_small, _,  _, _ = load_ckpt('/kaggle/working/Metric3D/weight/metric_depth_vit_small_800k.pth', model_small, strict_match=False)

from mono.utils.do_test import transform_test_data_scalecano
device = "cuda"
# model_large.to(device)
model.to(device)

def predict_depth_normal(img, model_selection="vit-small", fx=1000.0, fy=1000.0, state_cache={}):

    
    
    if img is None:
        return None, None, None, None, state_cache, "Please upload an image and wait for the upload to complete."


    cv_image = np.array(img) 
    img = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
    intrinsic = [fx, fy, img.shape[1]/2, img.shape[0]/2]
    rgb_input, cam_models_stacks, pad, label_scale_factor = transform_test_data_scalecano(img, intrinsic, cfg.data_basic)

    with torch.no_grad():
        pred_depth, pred_depth_scale, scale, output, confidence = get_prediction(
                    model = model,
                    input = rgb_input,
                    cam_model = cam_models_stacks,
                    pad_info = pad,
                    scale_info = label_scale_factor,
                    gt_depth = None,
                    normalize_scale = cfg.data_basic.depth_range[1],
                    ori_shape=[img.shape[0], img.shape[1]],
                )
        print(output['prediction_normal'].shape)
        
        pred_normal = output['prediction_normal'][:, :3, :, :] 
        H, W = pred_normal.shape[2:]
        pred_normal = pred_normal[:, :, pad[0]:H-pad[1], pad[2]:W-pad[3]]

    pred_depth = pred_depth.squeeze().cpu().numpy()
    pred_depth[pred_depth<0] = 0
    pred_color = gray_to_colormap(pred_depth)

    pred_normal = torch.nn.functional.interpolate(pred_normal, [img.shape[0], img.shape[1]], mode='bilinear').squeeze()
    pred_normal = pred_normal.permute(1,2,0)
    pred_color_normal = vis_surface_normal(pred_normal)
    pred_normal = pred_normal.cpu().numpy()
    
    # Storing depth and normal map in state for potential 3D reconstruction
    state_cache['depth'] = pred_depth
    state_cache['normal'] = pred_normal
    state_cache['img'] = img
    state_cache['intrinsic'] = intrinsic
    state_cache['confidence'] = confidence 

    # save depth and normal map to .npy file
    if 'save_dir' not in state_cache:
        cache_id = np.random.randint(0, 100000000000)
        while osp.exists(f'recon_cache/{cache_id:08d}'):
            cache_id = np.random.randint(0, 100000000000)
        state_cache['save_dir'] = f'recon_cache/{cache_id:08d}'
        os.makedirs(state_cache['save_dir'], exist_ok=True)
    depth_file = f"{state_cache['save_dir']}/depth.npy"
    normal_file = f"{state_cache['save_dir']}/normal.npy"
    np.save(depth_file, pred_depth)
    np.save(normal_file, pred_normal)

    ##formatted = (output * 255 / np.max(output)).astype('uint8')
    img = Image.fromarray(pred_color)
    img_normal = Image.fromarray(pred_color_normal)
    return img, output, img_normal, normal_file, state_cache, "Success!"

def get_camera(img):
    if img is None:
        return None, None, None, "Please upload an image and wait for the upload to complete."
    try:
        exif = img.getexif()
        exif.update(exif.get_ifd(ExifTags.IFD.Exif))
    except:
        exif = {}
    sensor_width = exif.get(ExifTags.Base.FocalPlaneYResolution, None)
    sensor_height = exif.get(ExifTags.Base.FocalPlaneXResolution, None)
    focal_length = exif.get(ExifTags.Base.FocalLength, None)
    
    # convert sensor size to mm, see https://photo.stackexchange.com/questions/40865/how-can-i-get-the-image-sensor-dimensions-in-mm-to-get-circle-of-confusion-from
    w, h = img.size
    sensor_width = w / sensor_width * 25.4 if sensor_width is not None else None
    sensor_height = h / sensor_height * 25.4 if sensor_height is not None else None
    focal_length = focal_length * 1.0 if focal_length is not None else None

    message = "Success!"
    if focal_length is None:
        message = "Focal length not found in EXIF. Please manually input."
    elif sensor_width is None and sensor_height is None:
        sensor_width = 16
        sensor_height = h / w * sensor_width
        message = f"Sensor size not found in EXIF. Using {sensor_width}x{sensor_height:.2f} mm as default."

    return sensor_width, sensor_height, focal_length, message

def get_intrinsic(img, sensor_width, sensor_height, focal_length):
    if img is None:
        return None, None, "Please upload an image and wait for the upload to complete."
    if sensor_width is None or sensor_height is None or focal_length is None:
        return 1000, 1000, "Insufficient information. Try detecting camera first or use default 1000 for fx and fy."
    if sensor_width == 0 or sensor_height == 0 or focal_length == 0:
        return 1000, 1000, "Insufficient information. Try detecting camera first or use default 1000 for fx and fy."
    
    # calculate focal length in pixels
    w, h = img.size
    fx = w / sensor_width * focal_length if sensor_width is not None else None
    fy = h / sensor_height * focal_length if sensor_height is not None else None

    # if fx is None:
    #     return fy, fy, "Sensor width not provided, using fy for both fx and fy"
    # if fy is None:
    #     return fx, fx, "Sensor height not provided, using fx for both fx and fy"

    return fx, fy, "Success!"


def unprojection_pcd(state_cache):
    depth_map = state_cache.get('depth', None)
    normal_map = state_cache.get('normal', None)
    img = state_cache.get('img', None)
    intrinsic = state_cache.get('intrinsic', None)

    if depth_map is None or img is None:
        return None, "Please predict depth and normal first."
    
    # # downsample/upsample the depth map to confidence map size
    # confidence = state_cache.get('confidence', None)
    # if confidence is not None:
    #     H, W = confidence.shape
    #     # intrinsic[0] *= W / depth_map.shape[1]
    #     # intrinsic[1] *= H / depth_map.shape[0]
    #     # intrinsic[2] *= W / depth_map.shape[1]
    #     # intrinsic[3] *= H / depth_map.shape[0]
    #     depth_map = cv2.resize(depth_map, (W, H), interpolation=cv2.INTER_LINEAR)
    #     img = cv2.resize(img, (W, H), interpolation=cv2.INTER_LINEAR)
    
    #     # filter out depth map by confidence
    #     mask = confidence.cpu().numpy() > 0

    # downsample the depth map if too large
    if depth_map.shape[0] > 1080:
        scale = 1080 / depth_map.shape[0]
        depth_map = cv2.resize(depth_map, (0, 0), fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
        img = cv2.resize(img, (0, 0), fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
        intrinsic = [intrinsic[0]*scale, intrinsic[1]*scale, intrinsic[2]*scale, intrinsic[3]*scale,intrinsic[4]*scale]
    
    if 'save_dir' not in state_cache:
        cache_id = np.random.randint(0, 100000000000)
        while osp.exists(f'recon_cache/{cache_id:08d}'):
            cache_id = np.random.randint(0, 100000000000)
        state_cache['save_dir'] = f'recon_cache/{cache_id:08d}'
        os.makedirs(state_cache['save_dir'], exist_ok=True)

    pcd_ply = f"{state_cache['save_dir']}/output.ply"
    pcd_obj = pcd_ply.replace(".ply", ".obj")
    
    pcd = reconstruct_pcd(depth_map, intrinsic[0], intrinsic[1], intrinsic[2], intrinsic[3])
    # if mask is not None:
    #     pcd_filtered = pcd[mask]
    #     img_filtered = img[mask]
    pcd_filtered = pcd.reshape(-1, 3)
    img_filtered = img.reshape(-1, 3)

    save_point_cloud(pcd_filtered, img_filtered, pcd_ply, binary=False)
    # ply_to_obj(pcd_ply, pcd_obj)

    # downsample the point cloud for visualization
    num_samples = 250000
    if pcd_filtered.shape[0] > num_samples:
        indices = np.random.choice(pcd_filtered.shape[0], num_samples, replace=False)
        pcd_downsampled = pcd_filtered[indices]
        img_downsampled = img_filtered[indices]
    else:
        pcd_downsampled = pcd_filtered
        img_downsampled = img_filtered

    # plotly show
    color_str = np.array([f"rgb({r},{g},{b})" for b,g,r in img_downsampled])
    data=[go.Scatter3d(
        x=pcd_downsampled[:,0],
        y=pcd_downsampled[:,1],
        z=pcd_downsampled[:,2],
        mode='markers',
        marker=dict(
            size=1,
            color=color_str,
            opacity=0.8,
        )
    )]
    layout = go.Layout(
        margin=dict(l=0, r=0, b=0, t=0),
        scene=dict(
            camera = dict(
                eye=dict(x=0, y=0, z=-1),
                up=dict(x=0, y=-1, z=0)
            ),
            xaxis=dict(showgrid=False, showticklabels=False, visible=False),
            yaxis=dict(showgrid=False, showticklabels=False, visible=False),
            zaxis=dict(showgrid=False, showticklabels=False, visible=False),
        )
    )
    fig = go.Figure(data=data, layout=layout)

    return fig, pcd_ply, "Success!"




# Step 4: Use Ngrok to expose the Gradio app
# public_url = ngrok.connect(7866)
# print(f"Public URL: {public_url}")

In [16]:
!pip install flask-cors


In [17]:
import plotly.graph_objects as go
import open3d as o3d
def unprojection_pcd(state_cache):
    depth_map = state_cache.get('depth', None)
    normal_map = state_cache.get('normal', None)
    img = state_cache.get('img', None)
    intrinsic = state_cache.get('intrinsic', None)

    if depth_map is None or img is None:
        return None, "Please predict depth and normal first."
    
    if depth_map.shape[0] > 1080:
        scale = 1080 / depth_map.shape[0]
        depth_map = cv2.resize(depth_map, (0, 0), fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
        img = cv2.resize(img, (0, 0), fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
        intrinsic = [intrinsic[0]*scale, intrinsic[1]*scale, intrinsic[2]*scale, intrinsic[3]*scale]
    
    if 'save_dir' not in state_cache:
        cache_id = np.random.randint(0, 100000000000)
        while osp.exists(f'recon_cache/{cache_id:08d}'):
            cache_id = np.random.randint(0, 100000000000)
        state_cache['save_dir'] = f'recon_cache/{cache_id:08d}'
        os.makedirs(state_cache['save_dir'], exist_ok=True)

    pcd_ply = f"{state_cache['save_dir']}/output.ply"
    pcd_obj = pcd_ply.replace(".ply", ".obj")
    print(depth_map.shape,type(depth_map))
    pcd = reconstruct_pcd(depth_map, intrinsic[0], intrinsic[2], intrinsic[3])
    # if mask is not None:
    #     pcd_filtered = pcd[mask]
    #     img_filtered = img[mask]
    print(pcd.shape,img.shape)
    pcd_filtered = pcd.reshape(-1, 3)
    img_filtered = img.reshape(-1, 3)
    # print(pcd_filtered.size,img_filtered.size)

    # save_point_cloud(pcd_filtered, img_filtered, pcd_ply, binary=False)
    # ply_to_obj(pcd_ply, pcd_obj)

    # downsample the point cloud for visualization
    num_samples = 250000
    if pcd_filtered.shape[0] > num_samples:
        indices = np.random.choice(pcd_filtered.shape[0], num_samples, replace=False)
        pcd_downsampled = pcd_filtered[indices]
        img_downsampled = img_filtered[indices]
    else:
        pcd_downsampled = pcd_filtered
        img_downsampled = img_filtered


    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcd_downsampled)
    # Normalize color values to [0, 1] if they are in [0, 255]
    pcd.colors = o3d.utility.Vector3dVector(img_downsampled / 255.0)
    
    # Save as a PLY file (Blender can import this)
    o3d.io.write_point_cloud(pcd_ply, pcd)
    
    return  pcd_ply, "Success!"

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [18]:
# Install pyngrok if not already installed
!pip install pyngrok

# Import ngrok
from pyngrok import ngrok


In [19]:
import os
import io
import zipfile
import torch
import torchvision.transforms as transforms
from flask import Flask, request, jsonify, send_file
from PIL import Image
from flask_cors import CORS

import shutil
public_url = ngrok.connect(5002)
print(f"Public URL: {public_url}") 


def move_ply(src_path, dest_folder="/kaggle/working/processed_folder"):

    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
    
    # Get the base name (file or folder name)
    item_name = os.path.basename(src_path)
    dest_path = os.path.join(dest_folder, item_name)
    
    # Move the file or folder
    shutil.move(src_path, dest_path)
    return dest_path

def save_pil_image(image,filename,folder="/kaggle/working/processed_folder"):
    """Save a tensor as a PNG image."""
    if not os.path.exists(folder):
        os.makedirs(folder)
    image.save(os.path.join(folder, filename))
    

def save_tensor_image(tensor, filename, folder="/kaggle/working/processed_folder"):
    """Save a tensor as a PNG image."""
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    # Convert tensor to PIL image
    transform = transforms.ToPILImage()
    image = transform(tensor.cpu().detach())
    
    # Save the image
    image.save(os.path.join(folder, filename))

def process_image(image):
    """Mock function to generate processed images."""
    img, output, img_normal, normal_file, state_cache, message = predict_depth_normal(image)
    pcd_ply, pcd_message = unprojection_pcd(state_cache)
    move_ply(pcd_ply)
    # if message != "success":
    #     return "error"
    
    # Save images
    save_tensor_image(output["roughness"].squeeze(0).squeeze(0), "roughness.png")
    save_tensor_image(output["ao"].squeeze(0).squeeze(0), "ambient_occlusion.png")
    save_pil_image(img_normal, "normal_map.png")
    save_pil_image(img, "depth.png")

    
    return "success"

app = Flask(__name__)
PROCESSED_FOLDER = "/kaggle/working/processed_folder"
CORS(app, resources={r"/*": {"origins": "*"}})
@app.route("/process", methods=["POST"])
def process():
    if "image" not in request.files:
        return jsonify({"error": "No image uploaded"}), 400
    
    image_file = request.files["image"]
    image = Image.open(image_file)
    save_path = os.path.join("/kaggle/working", image_file.filename)
    
    # Save the image
    image.save(save_path)
    
    result = process_image(image)
    if result == "error":
        return jsonify({"error": "Processing failed"}), 500
    
    zip_buffer = io.BytesIO()
    with zipfile.ZipFile(zip_buffer, "w", zipfile.ZIP_DEFLATED) as zipf:
        for filename in os.listdir(PROCESSED_FOLDER):
            if filename.endswith(".png") or filename.endswith(".ply"):  # Include depth, normal, and PLY
                zipf.write(os.path.join(PROCESSED_FOLDER, filename), filename)
    zip_buffer.seek(0)
    
    return send_file(zip_buffer, mimetype="application/zip", as_attachment=True, download_name="processed_images.zip")

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5002, debug=True, use_reloader=False)

ERROR:  authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.
ERROR:  Run multiple endpoints at the same time from a single agent by defining them in your agent configuration file and running `ngrok start --all`.
ERROR:  Read more about the agent configuration file: https://ngrok.com/docs/agent/config/ 
ERROR:  You can view your current agent sessions in the dashboard: https://dashboard.ngrok.com/agents. Upgrade to a paid plan to remove this limit:
ERROR:  https://dashboard.ngrok.com/billing/choose-a-plan
ERROR:  
ERROR:  ERR_NGROK_108
ERROR:  https://ngrok.com/docs/errors/err_ngrok_108
ERROR:  


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nRun multiple endpoints at the same time from a single agent by defining them in your agent configuration file and running `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/agent/config/ \nYou can view your current agent sessions in the dashboard: https://dashboard.ngrok.com/agents. Upgrade to a paid plan to remove this limit:\nhttps://dashboard.ngrok.com/billing/choose-a-plan\r\n\r\nERR_NGROK_108\r\n.